# Using the FMI time series interface

https://github.com/fmidev/smartmet-plugin-timeseries/wiki/Using-the-Timeseries-API

https://github.com/fmidev/smartmet-plugin-timeseries/wiki/Examples

https://opendata.fmi.fi/timeseries?producer=airquality_urban&area=Helsinki&param=name,time,fmisid,stationname,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg

https://opendata.fmi.fi/timeseries?format=ascii&groupareas=0&separator=,&attributes=time,fmisid&producer=airquality_urban&area=Helsinki&param=time,fmisid,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg


In [1]:
import requests
import datetime

In [12]:
end_time = datetime.datetime.utcnow()
start_time = end_time - datetime.timedelta(days=7)
start_time = start_time.isoformat(timespec="seconds")
end_time = end_time.isoformat(timespec="seconds")

display(start_time, end_time)

fields = [
    'name', 'time', 'fmisid', 'stationname', 
    'PM10_PT1H_avg', 'PM25_PT1H_avg', 
    'O3_PT1H_avg', 'CO_PT1H_avg', 'SO2_PT1H_avg', 
    'NO2_PT1H_avg', 'TRSC_PT1H_avg'
]

url = 'https://opendata.fmi.fi/timeseries'

params = {
    'format': 'json',
    'groupareas': '0',
    'separator': '|',
    'producer': 'airquality_urban',
    'area': 'Uusimaa',
    # 'attributes': ','.join(['time', 'fmisid']),
    'attributes': ','.join(['time']),
    'param': ','.join(fields),
    'starttime': start_time,
    'endtime': end_time,
    'tz': 'UTC',
}

data = requests.get(url, params=params).json()

'2022-07-30T20:47:13'

'2022-08-06T20:47:13'

In [6]:
#data = requests.get(url, params=params).json()

In [13]:
data.keys()

dict_keys(['20220730T210000', '20220730T220000', '20220730T230000', '20220731T000000', '20220731T010000', '20220731T020000', '20220731T030000', '20220731T040000', '20220731T050000', '20220731T060000', '20220731T070000', '20220731T080000', '20220731T090000', '20220731T100000', '20220731T110000', '20220731T120000', '20220731T130000', '20220731T140000', '20220731T150000', '20220731T160000', '20220731T170000', '20220731T180000', '20220731T190000', '20220731T200000', '20220731T210000', '20220731T220000', '20220731T230000', '20220801T000000', '20220801T010000', '20220801T020000', '20220801T030000', '20220801T040000', '20220801T050000', '20220801T060000', '20220801T070000', '20220801T080000', '20220801T090000', '20220801T100000', '20220801T110000', '20220801T120000', '20220801T130000', '20220801T140000', '20220801T150000', '20220801T160000', '20220801T170000', '20220801T180000', '20220801T190000', '20220801T200000', '20220801T210000', '20220801T220000', '20220801T230000', '20220802T000000', '